<a href="https://colab.research.google.com/github/sugampath/BERT-result/blob/master/full%20%20clean%20data%20final%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

     |████████████████████████████████| 2.1MB 7.2MB/s 
     |████████████████████████████████| 870kB 35.0MB/s 
     |████████████████████████████████| 3.3MB 49.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b7ed4ce23830c3383c26b0777903d685c0c70da236a3dc8c19995bf298abc9ba
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [3]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [4]:
data=pd.read_csv("/content/drive/MyDrive/Thesis/fulldatset.csv")

In [5]:
data.rename(columns = {"label":"category"}, inplace = True)

In [6]:
data.head(10)

,feature,category
0,मध्य पूर्वीय क्षेत्रको पहिलो नेपाली फेशन शो दु...,diaspora
1,एनआरएन नेदरल्याण्ड्सको अध्यक्षमा शर्माको उम्मे...,diaspora
2,राजदूत प्रतिभा राणाको सम्मानमा जापानमा स्वागत ...,diaspora
3,बिद्यार्थीहरुलाई लक्षित गरी टोकियोमा रोजगार मे...,diaspora
4,अस्ट्रियाबासी नेपालीहरुले क्यान्सरपीडित ५ बर्...,diaspora
5,मुक्तक मेलाको भव्य तयारीमा कोरिया\n\nसौल । अने...,diaspora
6,नेपाली जनसम्पर्क समिति क्यानडाका उपसभापति एवं ...,diaspora
7,विश्व नेपाली साहित्य महासंघ इजरायलको आयोजनामा ...,diaspora
8,वासिंगटन डिसीमा आनी लाईभ कन्सर्ट सम्पन्न\n\nवा...,diaspora
9,विश्व शरणार्थी दिवसको अवसरमा भलिबल प्रतियोगिता...,diaspora


In [7]:
data["feature"][200]

'खोटाङका रुद्र शर्मा आचार्यको एनआरएनए एनसिसी कतारको खुल्ला सदस्यमा उम्मेदवार\n\nरुद्र शर्मा, कतार । खोटाङ सेवा समाज कतारका द्दितिय उपाध्यक्ष्य रुद्र शर्मा आचार्यले गैरआवासीय नेपाली संघ कतारको २०१७–१९ कार्यकालको निर्वाचनका लागि खुल्ला सदस्य तर्फ औपचारिक रुपमा उम्मेदवार दिएका छन् ।\n\nकतारकै सबै भन्दा पुरानो र पहिलो नेपाली सस्था खोटाङ सेवा समाजमा कार्यसमिति सदश्य, सचिब र दुइ कार्यकाल महासचिब जस्तो जिम्बेवारी पदमा रही कार्यकाल सकुसल निर्बाह गरिसकेका र केहि समय पहिले NCC कतारको यहि २०१७–१९ कार्यकालको लागि सचिब पदका लागि उमेदवारी दिने मनासय राखे पनि आफ्ना केहि सहपाठी र सह यात्री साथीहरुलाई समर्थन गर्दै संस्थाको परिमार्जित विधानको सम्मान गर्दै सोही मूल्य मान्यता लाई आत्मसात गर्दै खुल्ला सदस्यमा उम्मेदवारी दिएका छन् ।\n\nकतारमा कार्यरत नेपाली श्रमिकहरुको अभिभाकत्व निभाइरहेको साझा संस्थामा रहेर नेपाली श्रमिकका हक हितका लागि काम गर्ने आफ्नो दृढ इच्छा रहेको आचार्य बताउँछन । उसले जति पनि पार्षद सदश्यहरु बनेका छन् उनीहरुको साथ, सहयोग र बिस्वासकै कारण आफु निर्वाचित हुनेमा विश्वस्त रहेको जिकिर समेत 

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
stop_words = stopwords.words("nepali")
data["feature"] = data["feature"].apply(lambda x: re.sub('[।(),०-९<<?!,—,–,/,’,‘,:,\u200d,\n,\xa0]', '', x))
data["feature"] = data["feature"].apply(lambda x: " ".join([i for i in x.split()if i not in (stop_words)]))

In [10]:
data["feature"][200]

'खोटाङका रुद्र शर्मा आचार्यको एनआरएनए एनसिसी कतारको खुल्ला सदस्यमा उम्मेदवाररुद्र शर्मा कतार खोटाङ सेवा समाज कतारका द्दितिय उपाध्यक्ष्य रुद्र शर्मा आचार्यले गैरआवासीय नेपाली संघ कतारको कार्यकालको निर्वाचनका खुल्ला सदस्य तर्फ औपचारिक रुपमा उम्मेदवार दिएका कतारकै पुरानो नेपाली सस्था खोटाङ सेवा समाजमा कार्यसमिति सदश्य सचिब दुइ कार्यकाल महासचिब जिम्बेवारी पदमा कार्यकाल सकुसल निर्बाह गरिसकेका केहि NCC कतारको यहि कार्यकालको सचिब पदका उमेदवारी दिने मनासय आफ्ना केहि सहपाठी सह यात्री साथीहरुलाई समर्थन संस्थाको परिमार्जित विधानको सम्मान मूल्य मान्यता आत्मसात खुल्ला सदस्यमा उम्मेदवारी दिएका कतारमा कार्यरत नेपाली श्रमिकहरुको अभिभाकत्व निभाइरहेको साझा संस्थामा रहेर नेपाली श्रमिकका हक हितका काम दृढ इच्छा आचार्य बताउँछन उसले जति पार्षद सदश्यहरु बनेका उनीहरुको सहयोग बिस्वासकै कारण आफु निर्वाचित हुनेमा विश्वस्त जिकिर समेत सस्थाको पदनै ठुलो नमानी सुकै पदमा रहे पद काम ठुलो मान्याता राख्दै बिगतमा बढी तँ मन दिई आगामी दिनमा सस्थाको सम्पूर्ण पदाधिकारी सँग हातेमालो योजना कार्यक्रममा प्रत्यक्ष सहभागी भइ सस्थाल

In [11]:
data.shape

(71970, 2)

In [12]:
data["category"].value_counts()

economy          14069
sports           13018
international     9879
politics          8352
entertainment     7588
diaspora          6224
society           4979
health            3122
opinion           2675
technology        2064
Name: category, dtype: int64

In [13]:
possible_labels = data.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'diaspora': 0,
 'economy': 1,
 'entertainment': 2,
 'health': 3,
 'international': 4,
 'opinion': 5,
 'politics': 6,
 'society': 7,
 'sports': 8,
 'technology': 9}

In [14]:
possible_labels

array(['diaspora', 'economy', 'entertainment', 'health', 'international',
       'opinion', 'politics', 'society', 'sports', 'technology'],
      dtype=object)

In [15]:
data["label"]=data.category.replace(label_dict)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

In [18]:
data["data_type"]=["not_set"]*data.shape[0]
data.loc[X_train,"data_type"]="train"
data.loc[X_val,"data_type"]="Val"

In [19]:
data.groupby(["category","label","data_type"]).count()

feature
category      label data_type         
diaspora      0     Val           1867
                    train         4357
economy       1     Val           4221
                    train         9848
entertainment 2     Val           2276
                    train         5312
health        3     Val            937
                    train         2185
international 4     Val           2964
                    train         6915
opinion       5     Val            802
                    train         1873
politics      6     Val           2506
                    train         5846
society       7     Val           1494
                    train         3485
sports        8     Val           3905
                    train         9113
technology    9     Val            619
                    train         1445

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [21]:
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].feature.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt')


encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='Val'].feature.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt')

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='Val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [22]:
a=tokenizer.tokenize(data["feature"][996])
print(a)
print(len(a))
print(a[:510])


['क', '##्या', '##ना', '##डा', '##मा', 'क', '##ु', '##द', '##ेर', 'दो', '##धार', '##ा', '##को', 'क', '##्या', '##म्', '##पस', '##ला', '##र', '##्', '##इ', 'स', '##ह', '##योग', 'ज', '##ुट', '##ा', '##उ', '##ँ', '##द', '##ै', 'भ', '##ु', '##प', '##ेश', 'ख', '##ड', '##्क', '##ा', '##क', '##्या', '##न', '##डा', 'क', '##्या', '##ना', '##डा', '##को', 'क', '##्या', '##ल', '##गर', '##ी', 'न', '##िव', '##ास', '##ी', 'भ', '##ु', '##प', '##ेश', 'ख', '##ड', '##्क', '##ा', 'नेपालको', 'क', '##ंच', '##न', '##पुर', 'जिल्लाको', '"', 'म', '##हा', '##काल', '##ी', 'प', '##ारी', '##को', 'नेपाल', '"', 'नाम', '##ले', 'पर', '##ि', '##चित', 'ब', '##िक', '##ट', 'दो', '##धार', '##ाचा', '##ँ', '##दन', '##ी', '##स', '##्थ', '##ित', 'भ', '##ान', '##ु', 'ब', '##ह', '##ु', '##म', '##ुख', '##ी', 'क', '##्या', '##म्', '##पस', '##को', 'पुस्तक', '##ालय', 'निर्माण', '##मा', 'स', '##घ', '##ा', '##उन', '##को', '"', 'हा', '##फ', 'म', '##्या', '##रा', '##थ', '##ुन', '"', 'स', '##ह', '##भा', '##गी', 'ह', '##ु', '##ँ', '##द', '

In [23]:
encoded_data_train

{'input_ids': tensor([[  101, 21650, 57863,  ..., 34028,   877,   102],
        [  101, 26876,   872,  ...,     0,     0,     0],
        [  101, 37245, 26876,  ...,     0,     0,     0],
        ...,
        [  101,   886, 69684,  ...,     0,     0,     0],
        [  101,   898, 36471,  ..., 97571, 13718,   102],
        [  101,   886, 90434,  ...,   896, 13671,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [24]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [25]:
len(dataset_train),len(dataset_val)

(50379, 21591)

In [26]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [27]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [28]:

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [29]:
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [30]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [31]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [33]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [34]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    FILE="model.pth"   
    torch.save(model.state_dict(),FILE)
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5829503460988927
Validation loss: 0.4010079928853239
F1 Score (Weighted): 0.9258460677392589



Epoch 2
Training loss: 0.32721054551452855
Validation loss: 0.36731781893161064
F1 Score (Weighted): 0.9406564103424109



Epoch 3
Training loss: 0.20319883286013388
Validation loss: 0.34835941661387004
F1 Score (Weighted): 0.946293516718771



Epoch 4
Training loss: 0.12233069287110125
Validation loss: 0.3556476858520341
F1 Score (Weighted): 0.9478621238926267



In [35]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [36]:
model.load_state_dict(torch.load('/content/model.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [37]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [38]:
accuracy_per_class(predictions, true_vals)

Class: diaspora
Accuracy: 1746/1867

Class: economy
Accuracy: 4030/4221

Class: entertainment
Accuracy: 2165/2276

Class: health
Accuracy: 835/937

Class: international
Accuracy: 2866/2964

Class: opinion
Accuracy: 675/802

Class: politics
Accuracy: 2361/2506

Class: society
Accuracy: 1341/1494

Class: sports
Accuracy: 3861/3905

Class: technology
Accuracy: 588/619



In [39]:
predlabel=np.argmax(predictions,axis=1)

In [40]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(true_vals,predlabel,digits =3))


              precision    recall  f1-score   support

           0      0.936     0.935     0.936      1867
           1      0.939     0.955     0.947      4221
           2      0.944     0.951     0.947      2276
           3      0.949     0.891     0.919       937
           4      0.958     0.967     0.963      2964
           5      0.918     0.842     0.878       802
           6      0.951     0.942     0.947      2506
           7      0.891     0.898     0.894      1494
           8      0.990     0.989     0.989      3905
           9      0.910     0.950     0.930       619

    accuracy                          0.948     21591
   macro avg      0.939     0.932     0.935     21591
weighted avg      0.948     0.948     0.948     21591



In [41]:
print(confusion_matrix(true_vals,predlabel))

[[1746   21   28    0   21   21    2   17    8    3]
 [  21 4030   16    9   11    3   42   65   11   13]
 [  28   27 2165    0   22   14    6    8    4    2]
 [   2   19    4  835   22    5    9   31    0   10]
 [   7   12   26    2 2866    3    7   11    9   21]
 [  27   15   17   13   12  675   27    9    2    5]
 [   3   74   10    7   13   11 2361   22    5    0]
 [  22   63   21   14    5    0   25 1341    1    2]
 [   9   12    7    0   10    0    3    1 3861    2]
 [   0   19    0    0    9    3    0    0    0  588]]
